# Analyze the Voltage and Current Dynamics of a CUBA LIF Neuron and choose appropriate values for each use case

### Check WD (change if necessary) and file loading

In [39]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/dynamics_analysis" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/dynamics_analysis"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

/home/monkin/Desktop/feup/thesis/thesis-lava/src/dynamics_analysis


## Channel Burst Detection 
Let's find the optimal parameters for the channel burst detection. we will **consider a network burst any sequence of spikes that occurs within 10 ms**.

Therefore, in the worst case scenario, the 2 spikes are separated by 10 ms.

To find the optimal parameters, we will test different combinations of the parameters: `spike_weight`, `du`, and `dv` and we will see the resulting voltage in the worst case scenario that we still consider as a burst (10 ms). To find the optimal parameters, we will also calculate the voltage if the spike occurs 1 time step delayed and choose the parameters where:
1. The voltage is above the threshold in the worst case scenario that is still considered a burst.
2. The voltage is below the threshold if the spike occurs 1 time step delayed.

In [40]:
from du_dv_analysis import voltage_pred, CUBADynamicsResult
import numpy as np

# Fixed Parameters
PADDING_TIME = 20               # 20ms padding time (to allow the voltage to reach the max value)
sim_time = 10 + PADDING_TIME    # 10ms simulation time (10 time steps)
num_spikes = 2                  # Number of spikes
spike_times = [0, 10]           # Let's consider the spikes are with maximum spacing
v_th = 1.0                      # Threshold voltage
recording_times = [10]

# Explorable Parameters
spike_weights = [0.05 * i for i in range(11)]
du_vals = [0.05 * i for i in range(1, 20)]
dv_vals = [0.05 * i for i in range(1, 20)]

# Variables to store the results
results_cond1 = []

# Run the simulation
for spike_weight in spike_weights:
    for du in du_vals:
        for dv in dv_vals:
            bef_v, v, after_v = voltage_pred(sim_time, du, dv, spike_times, recording_times, spike_weight)
            # print(f"bef_v: {bef_v}, v: {v}, after_v: {after_v}")

            currResult = CUBADynamicsResult(spike_weight, du, dv, bef_v, v, after_v)
            results_cond1.append(currResult)

In [41]:
# Show the results
print(results_cond1)

print("Number of Results Cond 1: ", len(results_cond1))

[{'spike_weight': 0.0, 'du': 0.05, 'dv': 0.05, 'before v': 0.0, 'spike v': 0.0, 'after v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.1, 'before v': 0.0, 'spike v': 0.0, 'after v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.15000000000000002, 'before v': 0.0, 'spike v': 0.0, 'after v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.2, 'before v': 0.0, 'spike v': 0.0, 'after v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.25, 'before v': 0.0, 'spike v': 0.0, 'after v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.30000000000000004, 'before v': 0.0, 'spike v': 0.0, 'after v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.35000000000000003, 'before v': 0.0, 'spike v': 0.0, 'after v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.4, 'before v': 0.0, 'spike v': 0.0, 'after v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.45, 'before v': 0.0, 'spike v': 0.0, 'after v': 0.0}, {'spike_weight': 0.0, 'du': 0.05, 'dv': 0.5, 'before v': 0.0, 'spike v': 0.0, 'after v': 0.0}

In [42]:
# Find the results where the Voltage is above 1.0 (i.e. a spike occurs
relevant_results1 = [result for result in results_cond1 if (
    result.bef_spike_v <= v_th and 
    result.spike_v >= v_th 
    # and result.after_spike_v <= v_th
    )]

# Print number of relevant results
print(f"Number of relevant results for Cond. 1: {len(relevant_results1)}")

# Sort the relevant results from lowest voltage to highest
relevant_results1 = sorted(relevant_results1, key=lambda item: item.after_spike_v, reverse=False)

# Show the relevant results
print(relevant_results1)

Number of relevant results for Cond. 1: 75
[{'spike_weight': 0.5, 'du': 0.1, 'dv': 0.4, 'before v': 0.5710530375000001, 'spike v': 1.01697104255, 'after v': 0.6992994628514534}, {'spike_weight': 0.5, 'du': 0.4, 'dv': 0.1, 'before v': 0.5710530375000001, 'spike v': 1.0169710425500003, 'after v': 0.6992994628514535}, {'spike_weight': 0.45, 'du': 0.15000000000000002, 'dv': 0.25, 'before v': 0.6325240033406249, 'spike v': 1.0129864844587937, 'after v': 0.7005388505631589}, {'spike_weight': 0.45, 'du': 0.25, 'dv': 0.15000000000000002, 'before v': 0.632524003340625, 'spike v': 1.012986484458794, 'after v': 0.700538850563159}, {'spike_weight': 0.5, 'du': 0.2, 'dv': 0.2, 'before v': 0.6710886400000005, 'spike v': 1.0905580032000004, 'after v': 0.7116147611837194}, {'spike_weight': 0.5, 'du': 0.15000000000000002, 'dv': 0.25, 'before v': 0.7028044481562498, 'spike v': 1.1255405382875485, 'after v': 0.7783765006257319}, {'spike_weight': 0.5, 'du': 0.25, 'dv': 0.15000000000000002, 'before v': 0.70

### Test the dynamics, but now with 1 more time step between the spikes

In [43]:
# Fixed Parameters
spike_times = [0, 11]           # Let's consider the spikes are with maximum spacing
recording_times = [10, 11]

# Variables to store the results
results_cond2 = []

# Run the simulation
for spike_weight in spike_weights:
    for du in du_vals:
        for dv in dv_vals:
            bef_v, v = voltage_pred(sim_time, du, dv, spike_times, recording_times, spike_weight)
            # print(f"bef_v: {bef_v}, v: {v}, after_v: {after_v}")

            currResult = CUBADynamicsResult(spike_weight, du, dv, bef_v, v, None)
            results_cond2.append(currResult)

In [44]:
# Find the results where the Voltage is below 1.0
relevant_results2 = [result for result in results_cond2 if (
    result.bef_spike_v <= v_th and 
    result.spike_v <= v_th
    )]

# Print number of relevant results
print(f"Number of relevant results Cond. 2: {len(relevant_results2)}")

# Sort the relevant results from lowest voltage to highest
relevant_results2 = sorted(relevant_results2, key=lambda item: item.spike_v, reverse=True)

# Show the relevant results
print(relevant_results2)

Number of relevant results Cond. 2: 3840
[{'spike_weight': 0.2, 'du': 0.05, 'dv': 0.15000000000000002, 'before v': 0.8029136971736911, 'spike v': 0.9962366610529294, 'after v': None}, {'spike_weight': 0.2, 'du': 0.15000000000000002, 'dv': 0.05, 'before v': 0.8029136971736911, 'spike v': 0.9962366610529293, 'after v': None}, {'spike_weight': 0.5, 'du': 0.6000000000000001, 'dv': 0.05, 'before v': 0.5170528629422362, 'spike v': 0.9912211913151243, 'after v': None}, {'spike_weight': 0.5, 'du': 0.05, 'dv': 0.6000000000000001, 'before v': 0.517052862942236, 'spike v': 0.9912211913151241, 'after v': None}, {'spike_weight': 0.45, 'du': 0.05, 'dv': 0.5, 'before v': 0.5683118110264598, 'spike v': 0.9901159470376368, 'after v': None}, {'spike_weight': 0.45, 'du': 0.5, 'dv': 0.05, 'before v': 0.5683118110264598, 'spike v': 0.9901159470376368, 'after v': None}, {'spike_weight': 0.4, 'du': 0.15000000000000002, 'dv': 0.2, 'before v': 0.6515511821569144, 'spike v': 0.9881782432013773, 'after v': None}

### See if the intersection between both solutions is not empty

In [46]:
relevant_results = []

for rel_result in relevant_results1:
    if rel_result in relevant_results2:
        # Get the index of the relevant result
        index = relevant_results2.index(rel_result)
        delayed_result = relevant_results2[index]

        relevant_results.append({
            "spike_weight": rel_result.spike_weight, "du": rel_result.du, "dv": rel_result.dv, 
            "spike_v_on_time": rel_result.spike_v, "spike_v_delayed": delayed_result.spike_v,
            "later_v_on_time": rel_result.after_spike_v, "bef_v_delayed": delayed_result.bef_spike_v,
        })

        # print(f"Parameters: {rel_result}")
        # print(f"On Time Results: {rel_result.to_dict()}")
        # print(f"Voltage delayed: {relevant_results2[index].to_dict()}\n")

# Sort the relevant results by lowest voltage later_v_on_time
relevant_results = sorted(relevant_results, key=lambda item: item["later_v_on_time"], reverse=False)

for rel_result in relevant_results:
    print(rel_result)

{'spike_weight': 0.5, 'du': 0.1, 'dv': 0.4, 'spike_v_on_time': 1.01697104255, 'spike_v_delayed': 0.9670879235750002, 'later_v_on_time': 0.6992994628514534, 'bef_v_delayed': 0.5169710425500001}
{'spike_weight': 0.5, 'du': 0.4, 'dv': 0.1, 'spike_v_on_time': 1.0169710425500003, 'spike_v_delayed': 0.9670879235750001, 'later_v_on_time': 0.6992994628514535, 'bef_v_delayed': 0.5169710425500001}
{'spike_weight': 0.45, 'du': 0.15000000000000002, 'dv': 0.25, 'spike_v_on_time': 1.0129864844587937, 'spike_v_delayed': 0.9475443230044217, 'later_v_on_time': 0.7005388505631589, 'bef_v_delayed': 0.5629864844587938}
{'spike_weight': 0.45, 'du': 0.25, 'dv': 0.15000000000000002, 'spike_v_on_time': 1.012986484458794, 'spike_v_delayed': 0.9475443230044218, 'later_v_on_time': 0.700538850563159, 'bef_v_delayed': 0.562986484458794}
{'spike_weight': 0.5, 'du': 0.6000000000000001, 'dv': 0.05, 'spike_v_on_time': 1.0170528629422362, 'spike_v_delayed': 0.9912211913151243, 'later_v_on_time': 0.8266543373877522, 'be

Looking at the results, we can see that the best parameters combination to detect the channel bursts with the above conditions are:

(**Dense Layer Weight**, **du**, **dv**):
- `(0.5, 0.1, 0.4)` or `(0.5, 0.4, 0.1)` or `(0.45, 0.15, 0.25)` or `(0.45, 0.25, 0.15)` 

Let's choose the parameters:
- `spike_weight = 0.45`
- `du = 0.25`
- `dv = 0.15`
as the optimal parameters and test it in a LAVA LIF Layer.